In [ ]:
testBitstring = '01001110100010010011'

In [ ]:
import itertools
import random
import math

In [ ]:
def bits_for_series(n):
    """
    Количество бит по Хартли:
    floor(log2(n!))
    """
    if n < 2:
        return 0
    return int(math.log2(math.factorial(n)))


In [ ]:
def split_into_blocks(data):
    """
    Возвращает список блоков:
    ('series', [элементы]) или ('term', элемент)
    """
    blocks = []
    current = []

    for x in data:
        if x not in current:
            current.append(x)
        else:
            if len(current) > 1:
                blocks.append(('series', current))
            else:
                blocks.append(('term', current[0]))
            blocks.append(('term', x))
            current = []

    if current:
        if len(current) > 1:
            blocks.append(('series', current))
        else:
            blocks.append(('term', current[0]))

    return blocks


In [ ]:
def get_allowed_permutations(series, salt):
    """
    Отбирает 2^k допустимых перестановок из n!
    """
    n = len(series)
    k = bits_for_series(n)
    count = 2 ** k

    all_perms = list(itertools.permutations(series))
    rnd = random.Random(salt + n)
    rnd.shuffle(all_perms)

    return all_perms[:count]


In [ ]:
def embed_series(series, secret_bits, salt):
    n = len(series)
    k = bits_for_series(n)

    if k == 0 or len(secret_bits) < k:
        return series

    # Серия длины 2 → 1 бит
    if n == 2:
        bit = secret_bits.pop(0)
        return series if bit == '0' else series[::-1]

    perms = get_allowed_permutations(series, salt)

    bits = ''.join(secret_bits[:k])
    del secret_bits[:k]

    idx = int(bits, 2)
    return list(perms[idx])


In [ ]:
def extract_series_bits(stego_series, original_series, salt):
    n = len(original_series)
    k = bits_for_series(n)

    if k == 0:
        return ""

    if n == 2:
        return '0' if stego_series == original_series else '1'

    perms = get_allowed_permutations(original_series, salt)
    idx = perms.index(tuple(stego_series))

    return format(idx, f'0{k}b')


In [ ]:
def SMT_embed(M, S, salt):
    secret_bits = list(S)
    blocks = split_into_blocks(M)

    output = []

    for kind, block in blocks:
        if kind == 'term':
            output.append(block)
            continue

        need = bits_for_series(len(block))

        if len(secret_bits) < need:
            output.extend(block)
            continue

        embedded = embed_series(block, secret_bits, salt)
        output.extend(embedded)

    return output


In [ ]:
def SMT_extract(M_original, M_stego, salt):
    blocks_orig = split_into_blocks(M_original)
    blocks_stego = split_into_blocks(M_stego)

    secret_bits = []

    for (k1, b1), (k2, b2) in zip(blocks_orig, blocks_stego):
        if k1 == 'series' and k2 == 'series' and len(b1) == len(b2):
            secret_bits.append(
                extract_series_bits(b2, b1, salt)
            )

    return ''.join(secret_bits)


In [ ]:
def SMT_capacity(M):
    blocks = split_into_blocks(M)
    cap = 0
    for kind, block in blocks:
        if kind == 'series':
            cap += bits_for_series(len(block))
    return cap

In [ ]:
M = [
    "00","01","10","11",
    "10","00","01","11",
    "00","10","01","11"
]
S = "1011001110001011"
salt = 12345
stego = SMT_embed(M, S, salt)
extracted = SMT_extract(M, stego, salt)
print("Capacity:", SMT_capacity(M), "bits")
print("Original: ", M)
print("Stego:    ", stego)
print("Secret:   ", S)
print("Extracted:", extracted[:len(S)])